In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

In [2]:
totalSpamWordCount=0
totalHamWordCount=0
file = open("DataSet.txt", "r")
content = file.read()


messages = {}
lines = content.strip().split('\n')
for index, line in enumerate(lines, start=1):
    label, text = line.split('\t', 1)
    text=nltk.word_tokenize(text)
    text = [word for word in text if word.lower() not in stop_words]
    if label=="spam":
        totalSpamWordCount+=len(text)
    else:
        totalHamWordCount+=len(text)
    text=(" ").join(text)
    messages[text]=label


    
dfTrainingData=pd.DataFrame({"Text":[],"Category":[]})

In [3]:
for key,value in messages.items():
    newrow=[key,value]
    dfTrainingData.loc[len(dfTrainingData)]=newrow

dfTrainingData

,Text,Category
0,"Go jurong point , crazy .. Available bugis n g...",ham
1,Ok lar ... Joking wif u oni ...,ham
2,Free entry 2 wkly comp win FA Cup final tkts 2...,spam
3,U dun say early hor ... U c already say ...,ham
4,"Nah n't think goes usf , lives around though",ham
...,...,...
5145,2nd time tried 2 contact u. U Â£750 Pound priz...,spam
5146,Ã¼ b going esplanade fr home ?,ham
5147,"Pity , * mood . ... suggestions ?",ham
5148,guy bitching acted like 'd interested buying s...,ham


In [4]:

Pspam=len(dfTrainingData[dfTrainingData["Category"]=="spam"])/len(dfTrainingData)
Pham=1-Pspam
print(Pspam,Pham)
print(Pspam)

0.12446601941747573 0.8755339805825243
0.12446601941747573


In [5]:
train_size = int(len(dfTrainingData) * 0.7)
test_size = len(dfTrainingData) - train_size

DataforTrain = dfTrainingData.iloc[:train_size]
DataforTest = dfTrainingData.iloc[train_size:]


In [6]:
SpamText=[]
HamText=[]

dham= DataforTrain[DataforTrain["Category"]=="ham"]["Text"]
for i in range(len(dham)):
    text=dham.iloc[i]
    text=nltk.word_tokenize(text)
    HamText+=text
    
dspam=DataforTrain[DataforTrain["Category"]=="spam"]["Text"]
for i in range(len(dspam)):
    text=dspam.iloc[i]
    text=nltk.word_tokenize(text)
    SpamText+=text

In [7]:
dfPrediction = pd.DataFrame(index=range(len(DataforTest)), columns=['Text', 'Category', 'Prediction'])

In [8]:
# Pre-compute word frequencies
spam_word_counts = {}
ham_word_counts = {}
for word in SpamText:
    spam_word_counts[word] = spam_word_counts.get(word, 0) + 1
for word in HamText:
    ham_word_counts[word] = ham_word_counts.get(word, 0) + 1



In [9]:
dfPrediction = pd.DataFrame(index=range(len(DataforTest)), columns=['Text', 'Category', 'Prediction'])

for i in range(len(DataforTest)):
    text = DataforTest.iloc[i]["Text"]
    cat = DataforTest.iloc[i]["Category"]
    listtext = nltk.word_tokenize(text)
    proSpam = 1
    proham = 1
    
    for j in listtext:
        if spam_word_counts.get(j,0)==0 or ham_word_counts.get(j,0)==0:
            continue
        proSpam = proSpam*(spam_word_counts.get(j)/len(SpamText))
        proham = proham*(ham_word_counts.get(j)/len(HamText))
    
    proham = proham*Pham
    proSpam = proSpam*Pspam
    
    dfPrediction.iloc[i] = [text, cat, 'spam' if proSpam > proham else 'ham']

In [10]:
dfPrediction[dfPrediction["Category"]!=dfPrediction["Prediction"]].head(10)

,Text,Category,Prediction
20,"Oh god ! 've found number ! 'm glad , text bac...",spam,ham
22,"THATÂ ’ ALRITE GIRL , U KNOW GAIL NEVA WRONG !...",ham,spam
39,link picture sent . also use http : //alto18.c...,spam,ham
50,Yes . Last practice,ham,spam
88,FIRST STRIKE RED ONE . BIRD + ANTELOPE BEGIN T...,ham,spam
102,Refused loan ? Secured Unsecured ? Ca n't get ...,spam,ham
126,CERI U REBEL ! SWEET DREAMZ LITTLE BUDDY ! ! C...,ham,spam
127,ringtoneking 84484,spam,ham
156,receiving week 's Triple Echo ringtone shortly...,spam,ham
175,Wot u 2 ? Thout u gon na call ! ! Txt bak luv K,ham,spam


In [11]:
hamham=len(dfPrediction[(dfPrediction["Category"]=="ham")&(dfPrediction["Prediction"]=="ham")])
hamspam=len(dfPrediction[(dfPrediction["Category"]=="ham")&(dfPrediction["Prediction"]=="spam")])
spamham=len(dfPrediction[(dfPrediction["Category"]=="spam")&(dfPrediction["Prediction"]=="ham")])
spamspam=len(dfPrediction[(dfPrediction["Category"]=="spam")&(dfPrediction["Prediction"]=="spam")])

In [12]:
precision=hamham/(hamham+spamham)

recall=hamham/(hamham+spamspam)

f1=(2*recall*precision)/(recall+precision)

accuracy=(hamham+spamham)/(hamham+spamham+hamspam+spamspam)

In [13]:
print("Recall: ", recall)
print("Precision: ",precision)
print("F1 score: ",f1)
print("Accuracy: ",accuracy)


Recall:  0.8945233265720081
Precision:  0.98
F1 score:  0.9353128313891834
Accuracy:  0.8732212160413971


In [14]:
def predict():
    text = input()
    listtext = nltk.word_tokenize(text)
    listtext=[word for word in listtext if word.lower() not in stop_words]
    proSpam = 1
    proham = 1
    
    for j in listtext:
        if spam_word_counts.get(j,0)==0 or ham_word_counts.get(j,0)==0:
            continue
        proSpam = proSpam*(spam_word_counts.get(j)/len(SpamText))
        proham = proham*(ham_word_counts.get(j)/len(HamText))
    
    proham = proham*Pham
    proSpam = proSpam*Pspam
    
    val= ['spam' if proSpam > proham else 'ham']
    print(val)

In [15]:
predict()

['ham']
